# Data Integration and Data Reshaping of Victorian Housing Data


## Table of Content
1. [Load and Parse files of different formats](#1)
2. [Data Integration](#2)
3. [Data Reshaping](#3)

## 1. Load and Parse data files of different formats <a class="anchor" id="1"></a>

## 2. Data Integration <a class="anchor" id="2"></a>

## 3. Data Reshaping <a class="anchor" id="3"></a>

In [ ]:
__